In [1]:
from __future__ import division

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import pyomo.environ as pyo, numpy as np, pandas as pd
from pyomo.environ import *
from pyomo.opt import SolverFactory


geradores = pd.read_excel('input.xlsx', sheet_name='geradores')
demanda   = pd.read_excel('input.xlsx', sheet_name='demanda')


In [2]:
demanda

,t,dem
0,0,355
1,1,480
2,2,650
3,3,758
4,4,791
5,5,885
6,6,899
7,7,1005
8,8,955
9,9,814


In [10]:
# Entrada de dados:
#
# Geradores
cos = geradores.Custo.values
a = geradores.A.values*cos
b = geradores.B.values*cos
c = geradores.C.values*cos
pmin = geradores.Pmin.values
pmax = geradores.Pmax.values
ur = geradores.UR.values
dr = geradores.DR.values
#cos = geradores.Custo.to_list()
#pmin = geradores.Pmin.to_list()
#pmax = geradores.Pmax.to_list()
#ur = geradores.UR.to_list()
#dr = geradores.DR.to_list()

# Demanda
Pd = demanda.dem.values
t = demanda.t.to_list()

Ng = len(geradores)
Nd = len(Pd)
c

array([0.001278, 0.00194 , 0.00482 ])

In [11]:
model = pyo.ConcreteModel()

model.Pg = pyo.Var(range(Ng), range(Nd), bounds=(0,None))
Pg = model.Pg

In [12]:
#objetivo

model.obj = pyo.Objective(expr= sum([a[g] + b[g]*Pg[g,t] + c[g]*Pg[g,t]*Pg[g,t] for g in range(Ng) for t in range(Nd)]), sense=minimize)

In [13]:
# balanco:
model.balanco = ConstraintList()
#sum_Pg = 0
for t in range(Nd):
    sum_Pg = sum(Pg[g,t] for g in range(Ng))
    model.balanco.add(expr= sum_Pg == Pd[t])

In [14]:
# limites geração:
model.limger = pyo.ConstraintList()
for g in range(Ng):
    for t in range(Nd):
        model.limger.add(inequality(pmin[g], Pg[g,t], pmax[g]))


In [17]:
opt = SolverFactory('couenne', executable='C:\\couenne\\bin\\couenne.exe')
#opt = SolverFactory('glpk')
opt.solve(model)

model.pprint()

5 Set Declarations
    Pg_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain                : Size : Members
        None :     2 : Pg_index_0*Pg_index_1 :   72 : {(0, 0), (0, 1), (0, 2), (0, 3), (0, 4), (0, 5), (0, 6), (0, 7), (0, 8), (0, 9), (0, 10), (0, 11), (0, 12), (0, 13), (0, 14), (0, 15), (0, 16), (0, 17), (0, 18), (0, 19), (0, 20), (0, 21), (0, 22), (0, 23), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (1, 11), (1, 12), (1, 13), (1, 14), (1, 15), (1, 16), (1, 17), (1, 18), (1, 19), (1, 20), (1, 21), (1, 22), (1, 23), (2, 0), (2, 1), (2, 2), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 8), (2, 9), (2, 10), (2, 11), (2, 12), (2, 13), (2, 14), (2, 15), (2, 16), (2, 17), (2, 18), (2, 19), (2, 20), (2, 21), (2, 22), (2, 23)}
    Pg_index_0 : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {0, 1, 2}
    Pg_index_1 : Size=1, Index=None, Ordered=False

In [18]:
for g in range(Ng):
    for t in range(Nd):
        print(f"Pg[{g},{t}] = {pyo.value(Pg[g,t])} / Pd[{t} == {pyo.value(Pd[t])}]")


Pg[0,0] = 204.99999999999997 / Pd[0 == 355]
Pg[0,1] = 330.0 / Pd[1 == 480]
Pg[0,2] = 500.0 / Pd[2 == 650]
Pg[0,3] = 600.0 / Pd[3 == 758]
Pg[0,4] = 600.0 / Pd[4 == 791]
Pg[0,5] = 600.0 / Pd[5 == 885]
Pg[0,6] = 600.0 / Pd[6 == 899]
Pg[0,7] = 600.0 / Pd[7 == 1005]
Pg[0,8] = 600.0 / Pd[8 == 955]
Pg[0,9] = 600.0 / Pd[9 == 814]
Pg[0,10] = 600.0 / Pd[10 == 920]
Pg[0,11] = 600.0 / Pd[11 == 1187]
Pg[0,12] = 600.0 / Pd[12 == 1037]
Pg[0,13] = 600.0 / Pd[13 == 1195]
Pg[0,14] = 600.0 / Pd[14 == 1148]
Pg[0,15] = 600.0 / Pd[15 == 1051]
Pg[0,16] = 600.0 / Pd[16 == 933]
Pg[0,17] = 600.0 / Pd[17 == 761]
Pg[0,18] = 553.0 / Pd[18 == 703]
Pg[0,19] = 495.0 / Pd[19 == 645]
Pg[0,20] = 239.99999999999997 / Pd[20 == 390]
Pg[0,21] = 171.99999999999997 / Pd[21 == 322]
Pg[0,22] = 229.0 / Pd[22 == 379]
Pg[0,23] = 243.99999999999997 / Pd[23 == 394]
Pg[1,0] = 100.00000000000001 / Pd[0 == 355]
Pg[1,1] = 100.00000000000001 / Pd[1 == 480]
Pg[1,2] = 100.0 / Pd[2 == 650]
Pg[1,3] = 108.0 / Pd[3 == 758]
Pg[1,4] = 140.999999